In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time
import asyncio

In [ ]:
!pip install playwright
!playwright install
!playwright install-deps
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.9/535.9 kB 24.5 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 2.0.2
    Uninstalling greenlet-2.0.2:
      Successfully uninstalled greenlet-2.0.2
144.3 Mb [] 0% 0.0s144.3 Mb [] 0% 29.8s144.3 Mb [] 0% 28.6s144.3 Mb [] 0% 22.3s144.3 Mb [] 0% 15.8s144.3 Mb [] 0% 11.2s144.3 Mb [] 1% 8.7s144.3 Mb [] 1% 7.0s144.3 Mb [] 2% 5.7s144.3 Mb [] 3% 5.2s144.3 Mb [] 3% 4.8s144.3 Mb [] 4% 4.4s144.3 Mb [] 5% 3.8s144.3 Mb [] 6% 3.7s144.3 Mb [] 6% 3.9s144.3 Mb [] 7% 3.5s144.3 Mb [] 8% 3.2s144.3 Mb [] 9% 3.1s144.3 Mb [] 10% 3.0s144.3 Mb [] 11% 2.8s144.3 Mb [] 12% 2.6s144.3 Mb [] 13% 2.4s144.3 Mb [] 14% 2.4s144.3 Mb [] 15% 2.2s144.3 Mb [] 17% 2.1s144.3 Mb [] 18% 2.0s144.3 Mb [] 19% 1.9s144.3 Mb [] 20% 1.8s144.3 Mb [] 21% 1.9s144.3 Mb [] 

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

Executing: /tmp/apt-key-gpghome.rbvKth9HA4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ocPSs3aZdw/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.5bJX6SJbrZ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
SEASONS = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [ ]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/project leo/data2/"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings2")
SCORES_DIR = os.path.join(DATA_DIR, "scores2")

NameError: ignored

In [ ]:
async def get_html(url, selector, sleep=0.1, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch_persistent_context(executable_path="/usr/bin/chromium", user_data_dir="/content/random-user")
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [ ]:
year = 2017
url = f"https://www.basketball-reference.com/leagues/NBA_{year}_games.html"

In [ ]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for season in SEASONS:
    await scrape_season(season)

NameError: ignored

In [ ]:
standings_files = os.listdir(STANDINGS_DIR)

In [ ]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

Bulls vs Rockets, March 11, 2023 | Basketball-Reference.com
Mavericks vs Grizzlies, March 11, 2023 | Basketball-Reference.com
Bucks vs Warriors, March 11, 2023 | Basketball-Reference.com
Thunder vs Pelicans, March 11, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202303110PHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202303110PHO.html
Timeout error on https://www.basketball-reference.com/boxscores/202303110PHO.html
Nets vs Nuggets, March 12, 2023 | Basketball-Reference.com
Cavaliers vs Hornets, March 12, 2023 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202303120PHI.html
Wizards vs 76ers, March 12, 2023 | Basketball-Reference.com
Trail Blazers vs Pelicans, March 12, 2023 | Basketball-Reference.com
Thunder vs Spurs, March 12, 2023 | Basketball-Reference.com
Knicks vs Lakers, March 12, 2023 | Basketball-Reference.com
Pacers vs Pistons, March 13, 2023 | Basketball-Ref

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
